In [151]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', 500)

In [152]:
train = pd.read_csv('train-flattened.csv',dtype={'date': str, 'fullVisitorId': str, 'sessionId':str, 'visitId': np.int64})

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [153]:
test = pd.read_csv('test-flattened.csv',dtype={'date': str, 'fullVisitorId': str, 'sessionId':str, 'visitId': np.int64})

In [154]:
revenue = train['totals.transactionRevenue']
revenue.isna().sum()/len(revenue)

0.9872572768529513

In [155]:
train['totals.transactionRevenue'] = train['totals.transactionRevenue'].fillna(0)

In [156]:
trainbrowser = train['device.browser']

In [157]:
trainbrowser = pd.get_dummies(trainbrowser)

In [158]:
trainbrowser.shape

(903653, 54)

In [159]:
testbrowser = test['device.browser']
testbrowser = pd.get_dummies(testbrowser)

In [160]:
concat = pd.concat((trainbrowser,testbrowser))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [161]:
trainbrowser1 = concat[:len(train)]
testbrowser1 = concat[len(train):]

In [162]:
trainbrowser1 = trainbrowser1.fillna(0)
testbrowser1 = testbrowser1.fillna(0)

In [163]:
#Make array for x and y values of training set
#x = train.drop['age']
xarraytrain = trainbrowser1.values

#ytrain = train['age']
yarraytrain = revenue.values

#create linear regression object
lm = LinearRegression()
#Fit regression model for training data
lm.fit(xarraytrain,yarraytrain)
predictions = lm.predict(testbrowser1)

In [188]:
predictions = pd.DataFrame(predictions)


In [189]:
ids = test["fullVisitorId"]
predictions['fullVisitorId'] = ids

In [190]:
preds = predictions[0].groupby(predictions['fullVisitorId']).sum()

In [191]:
preds = pd.DataFrame(preds)

In [192]:
preds = preds.reset_index()

In [193]:
preds.columns = ['fullVisitorId', 'PredictedLogRevenue']

In [195]:
preds['PredictedLogRevenue'] = preds['PredictedLogRevenue'].apply(np.log1p)

In [201]:
preds['PredictedLogRevenue'] = preds['PredictedLogRevenue'].fillna(0)

In [203]:
preds.to_csv('predictions3.csv', index = False)